- http://www.universalteacherpublications.com/univ/ebooks/or/index1.htm
- https://katex.org/docs/supported.html


# Queuing Theory (Waiting Line Models)

http://www.universalteacherpublications.com/univ/ebooks/or/Ch10/queintro.htm

## Assumptions of Queuing Theory
- The source population has infinite size.
- The inter-arrival time has an exponential probability distribution with a mean arrival rate of l customer arrivals per unit time.
- There is no unusual customer behaviour.
- The service discipline is FIFO.
- The service time has an exponential probability distribution with a mean service rate of m service completions per unit time.
- The mean arrival rate is less than the mean service rate, i.e., l < m.
- There is no unusual server behaviour.


## M/M/1 Queuing System (∞/FIFO)

It is a queuing model where the arrivals follow a Poisson process, service times are exponentially distributed and there is only one server. In other words, it is a system with Poisson input, exponential waiting time and Poisson output with single channel.

Queue capacity of the system is infinite with first in first out mode. The first M in the notation stands for Poisson input, second M for Poisson output, 1 for the number of servers and ∞ for infinite capacity of the system.

### Formulas

http://www.universalteacherpublications.com/univ/ebooks/or/Ch10/mm1ex.htm

 performance indicator                                    | Formula 
:---------------------------------------------------------|:--------
 Probability of zero unit in the queue ($P_o$)            | $1 - \cfrac{\lambda}{\mu}$
 Average queue length ($L_q$ )                            | $\cfrac{\lambda^2}{\mu(\mu-\lambda)}$
 Average number of units in the system ($L_s$)            | $\cfrac{\lambda}{\mu-\lambda}$
 Average waiting time of an arrival ($W_q$)               | $\cfrac{\lambda}{\mu(\mu-\lambda)}$
 Average waiting time of an arrival in the system ($W_s$) | $\cfrac{1}{\mu-\lambda}$

### Example 1

Cars arrive at a single charging station according to a Poisson input process with a mean rate of 40 per hour. The time required to charge a car has an exponential distribution with a mean of 50 per hour. Assume that the cars are charged by a single charging station, find the average waiting time of a car.

#### Solution

Given:

$\lambda = 40/hour$, $\mu = 50/hour$

Average waiting time of a car before receiving service ($W_q$) = $\cfrac{40}{50(50-40)}$ = 4.8 minutes


#### Simulation of Example 1

##### passivate/activate method

- [A bank example, with multiple clerks](https://www.salabim.org/manual/Modelling.html#a-bank-example)
- [sample models/Bank, 3 clerks.py (github)](https://github.com/salabim/salabim/blob/master/sample%20models/Bank%2C%203%20clerks.py)

###### Components:

- **CarGenerator:** generates cars with an exponential *inter arrival time* with a mean of 60/40 minutes resuting in 40 cars per hour according to a Possion distribution.
- **Car:** The car enters the waiting line and checks if any of the stations is available. 
In case it is the station is activated. 
- **ChargingStation:** Take the first car from the waiting line (FIFO), charge the car and activate the car as soon as the car is fully charged.  
- **Queue:** A waiting line for the cars waiting to be charged.

In [ ]:
# passivate/activate method
# https://www.salabim.org/manual/Modelling.html#a-bank-example

import salabim as sim


class CarGenerator(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        while True:
            Car()
            self.hold(iat_distr.sample())


class Car(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        self.enter(waitingline)
        for ChargingStation in ChargingStations:
            if ChargingStation.ispassive():
                ChargingStation.activate()
                break  # activate at most one charging station
        self.passivate()


class ChargingStation(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        while True:
            while len(waitingline) == 0:
                self.passivate()
            self.car = waitingline.pop()
            self.hold(srv_distr.sample())
            self.car.activate()


N_STATION = 1
iat_distr = sim.Exponential(60 / 40)
srv_distr = sim.Exponential(60 / 50)

# https://www.salabim.org/manual/Reference.html#environment
app = sim.App(
    trace=False,  # defines whether to trace or not
    random_seed="*",  # if “*”, a purely random value (based on the current time)
    time_unit="minutes",  # defines the time unit used in the simulation
    name="Charging Station",  # name of the simulation
    do_reset=True,  # defines whether to reset the simulation when the run method is called
    yieldless=True,  # defines whether the simulation is yieldless or not
)

# Instantiate and activate the client generator
CarGenerator(name="Electric Cars Generator")

# Create Queue and set monitor to stats_only
waitingline = sim.Queue(name="Waiting Cars", monitor=False)
waitingline.length_of_stay.monitor(value=True)
waitingline.length_of_stay.reset_monitors(stats_only=True)

# Instantiate the servers, list comprehension but only 1 server
ChargingStations = [ChargingStation() for _ in range(N_STATION)]

# Execute Simulation
app.run(till=50000)

# Print statistics
waitingline.length_of_stay.print_statistics()

##### Stores method

- [The bank office example with stores](https://www.salabim.org/manual/Modelling.html#the-bank-office-example-with-stores)
- [sample models/Bank, 3 clerks (store).py (github)](https://github.com/salabim/salabim/blob/master/sample%20models/Bank%2C%203%20clerks%20(store).py)

A store is essentially a queue (optionally with limited capacity) that can hold components.

And we can request components from a store. If there’s a component in the store, it is returned. But if it is not the requesting component goes into the requesting state, until something is available in the store.

The same holds for processes putting components in the store: if it is full, the component that want to add someting to the store goes into the requesting state. Here we have an unlimited waiting room, though.

###### Components:

- **CarGenerator:** generates cars with an exponential *inter arrival time* with a mean of 60/40 minutes resuting in 40 cars per hour according to a Possion distribution.
- **Car:** a data component without process
- **ChargingStation:** Take the first car from the waiting line (FIFO), charge the car and activate the car as soon as the car is fully charged.  
- **Store:** A waiting line for the cars waiting to be charged.

In [ ]:
# stores method
# https://www.salabim.org/manual/Modelling.html#the-bank-office-example-with-stores

import salabim as sim


class CarGenerator(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        while True:
            Car().enter(waitingline)
            self.hold(iat_distr.sample())


class Car(sim.Component):
    ...


class ChargingStation(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        while True:
            car = self.from_store(waitingline)
            self.hold(srv_distr.sample())


N_STATION = 1
iat_distr = sim.Exponential(60 / 40)
srv_distr = sim.Exponential(60 / 50)

# https://www.salabim.org/manual/Reference.html#environment
app = sim.App(
    trace=False,  # defines whether to trace or not
    random_seed="*",  # if “*”, a purely random value (based on the current time)
    time_unit="minutes",  # defines the time unit used in the simulation
    name="Charging Station",  # name of the simulation
    do_reset=True,  # defines whether to reset the simulation when the run method is called
    yieldless=True,  # defines whether the simulation is yieldless or not
)

# Instantiate and activate the client generator
CarGenerator(name="Electric Cars Generator")

# Create Store and set monitor to stats_only
waitingline = sim.Store(name="Waiting Cars")
waitingline.length.monitor(value=False)
waitingline.length_of_stay.reset_monitors(stats_only=True)

# Instantiate the servers, list comprehension but only 1 server
ChargingStations = [ChargingStation() for _ in range(N_STATION)]

# Execute Simulation
app.run(till=50000)

# Print statistics
waitingline.length_of_stay.print_statistics()

##### Resource method

- [The bank office example with resources](https://www.salabim.org/manual/Modelling.html#the-bank-office-example-with-resources)
- [sample models/Bank, 3 clerks (resources).py (github)](https://github.com/salabim/salabim/blob/master/sample%20models/Bank%2C%203%20clerks%20(resources).py)

Resources have a limited capacity and can be claimed by components and released later.

###### Components:

- **CarGenerator:** generates cars with an exponential *inter arrival time* with a mean of 60/40 minutes resuting in 40 cars per hour according to a Possion distribution.
- **Car:** a data component without process
- **ChargingStation:** Modelled as a resource  

In [ ]:
# Resource method
# https://www.salabim.org/manual/Modelling.html#the-bank-office-example-with-resources

import salabim as sim


class CarGenerator(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        while True:
            Car()
            self.hold(iat_distr.sample())


class Car(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        self.request(chargingstations)
        self.hold(srv_distr.sample())
        self.release()  # not really required

N_STATION = 1
iat_distr = sim.Exponential(60 / 40)
srv_distr = sim.Exponential(60 / 50)

# https://www.salabim.org/manual/Reference.html#environment
app = sim.App(
    trace=False,  # defines whether to trace or not
    random_seed="*",  # if “*”, a purely random value (based on the current time)
    time_unit="minutes",  # defines the time unit used in the simulation
    name="Charging Station",  # name of the simulation
    do_reset=True,  # defines whether to reset the simulation when the run method is called
    yieldless=True,  # defines whether the simulation is yieldless or not
)

# Instantiate and activate the client generator
CarGenerator(name="Electric Cars Generator")

# Instantiate the servers, list comprehension but only 1 server
chargingstations = sim.Resource(name="Charging Stations", capacity=N_STATION, monitor=False)
chargingstations.requesters().length_of_stay.monitor(value=True)

# Execute Simulation
app.run(till=50000)

# Print statistics
chargingstations.requesters().length_of_stay.print_statistics()

##### State method

- [The bank office example with states](https://www.salabim.org/manual/Modelling.html#the-bank-office-example-with-states)
- [sample models/Bank, 3 clerks (state).py (github)](https://github.com/salabim/salabim/blob/master/sample%20models/Bank%2C%203%20clerks%20(state).py)

States together with the Component.wait() method provide a powerful way of process interaction.

A state will have a certain value at a given time. In its simplest form a component can then wait for a specific value of a state. Once that value is reached, the component will be resumed.

We define a state called worktodo with initial value ```False```, meaning **no** work to do

###### Components:

- **CarGenerator:** generates cars with an exponential *inter arrival time* with a mean of 60/40 minutes resuting in 40 cars per hour according to a Possion distribution.
- **Car:** a data component without process
- **ChargingStation:** a class 
- **waitingline:** a queue 
- **worktodo:** a State

In [ ]:
# State method
# https://www.salabim.org/manual/Modelling.html#the-bank-office-example-with-states

import salabim as sim


class CarGenerator(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        while True:
            Car()
            self.hold(iat_distr.sample())


class Car(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        self.enter(waitingline)
        worktodo.trigger(max=1)
        self.passivate()


class ChargingStation(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        while True:
            if len(waitingline) == 0:
                self.wait((worktodo, True, 1))
            self.car = waitingline.pop()
            self.hold(srv_distr.sample())
            self.car.activate()


N_STATION = 1
iat_distr = sim.Exponential(60 / 40)
srv_distr = sim.Exponential(60 / 50)

# https://www.salabim.org/manual/Reference.html#environment
app = sim.App(
    trace=False,  # defines whether to trace or not
    random_seed="*",  # if “*”, a purely random value (based on the current time)
    time_unit="minutes",  # defines the time unit used in the simulation
    name="Charging Station",  # name of the simulation
    do_reset=True,  # defines whether to reset the simulation when the run method is called
    yieldless=True,  # defines whether the simulation is yieldless or not
)

# Instantiate and activate the client generator
CarGenerator(name="Electric Cars Generator")

# Create Queue and set monitor to stats_only
waitingline = sim.Queue(name="Waiting Cars", monitor=False)
waitingline.length_of_stay.monitor(value=True)
waitingline.length_of_stay.reset_monitors(stats_only=True)

# create State
worktodo = sim.State(name="worktodo", value=False)

# Instantiate the servers, list comprehension but only 1 server
ChargingStations = [ChargingStation() for _ in range(N_STATION)]

# Execute Simulation
app.run(till=50000)

# Print statistics
waitingline.length_of_stay.print_statistics()

print("\n")
worktodo.waiters().length_of_stay.print_statistics()

##### Standby method

- [The bank office example with standby](https://www.salabim.org/manual/Modelling.html#the-bank-office-example-with-standby)
- [sample models/Bank, 3 clerks (standby).py (github)](https://github.com/salabim/salabim/blob/master/sample%20models/Bank%2C%203%20clerks%20(standby).py)

When a component is in standby mode, it will become current after each event.  
Normally, the standby will be used in a while loop where at every event one or more conditions are checked, eg:

```
while len(waitingline) == 0:
    self.standby()
```

The rest of the code is very similar to the version with states.

!!!  
It is very important to realize that this mechanism can have significant impact on the performance, as after EACH event, the component becomes current and has to be checked. In general it is recommended to try and use states or a more straightforward passivate/activate construction.  
!!!  

###### Components:

- **CarGenerator:** generates cars with an exponential *inter arrival time* with a mean of 60/40 minutes resuting in 40 cars per hour according to a Possion distribution.
- **Car:** a data component without process
- **ChargingStation:** a class 
- **waitingline:** a queue 
- **worktodo:** a State

In [3]:
# Standby method
# https://www.salabim.org/manual/Modelling.html#the-bank-office-example-with-standby
# NOTE POTENTIAL PERFORMANCE IMPACT

import salabim as sim


class CarGenerator(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        while True:
            Car()
            self.hold(iat_distr.sample())


class Car(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        self.enter(waitingline)
        self.passivate()


class ChargingStation(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        while True:
            while len(waitingline) == 0:  # note the while loop
                self.standby()
            self.car = waitingline.pop()
            self.hold(srv_distr.sample())
            self.car.activate()


N_STATION = 1
iat_distr = sim.Exponential(60 / 40)
srv_distr = sim.Exponential(60 / 50)

# https://www.salabim.org/manual/Reference.html#environment
app = sim.App(
    trace=False,  # defines whether to trace or not
    random_seed="*",  # if “*”, a purely random value (based on the current time)
    time_unit="minutes",  # defines the time unit used in the simulation
    name="Charging Station",  # name of the simulation
    do_reset=True,  # defines whether to reset the simulation when the run method is called
    yieldless=True,  # defines whether the simulation is yieldless or not
)

# Instantiate and activate the client generator
CarGenerator(name="Electric Cars Generator")

# Create Queue and set monitor to stats_only
waitingline = sim.Queue(name="Waiting Cars", monitor=False)
waitingline.length_of_stay.monitor(value=True)
waitingline.length_of_stay.reset_monitors(stats_only=True)

# Instantiate the servers, list comprehension but only 1 server
ChargingStations = [ChargingStation() for _ in range(N_STATION)]

# Execute Simulation
app.run(till=50000)

# Print statistics
# waitingline.length_of_stay.print_statistics()
Wq = waitingline.length_of_stay.mean()

print(
    "\n",
    "Average waiting time of a car before being charged: \t",
    Wq, 
    " minutes."
)


 Average waiting time of a car before being charged: 	 4.529244568884807  minutes.


## Example 2

New Delhi Charging Hub has a single charging station. During the rush hours, customers arrive at the rate of 10 per hour. The average number of customers that can be served is 12 per hour. Find out the following:

1. Probability that the ticket counter is free.
1. Average number of customers in the queue.

### Solution

Given  

$\lambda = 10/hour$, $\mu = 12/hour$

1. Probability that the charging station is free ($P_o$) = $1 - \cfrac{\lambda}{\mu} = 1 - \cfrac{10}{12} = \cfrac{1}{6}$   

1. Average number of customers in the queue ($L_q$ ) = $\cfrac{\lambda^2}{\mu(\mu-\lambda)} = \cfrac{10^2}{12(12-10)} = \cfrac{25}{6} = 4.16666$


#### Simulation of Example 2

In [ ]:
# Resource method
# https://www.salabim.org/manual/Modelling.html#the-bank-office-example-with-resources

import salabim as sim


class CarGenerator(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        while True:
            Car()
            self.hold(iat_distr.sample())


class Car(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        self.request(chargingstations)
        self.hold(srv_distr.sample())
        self.release()  # not really required


N_STATION = 1
iat_distr = sim.Exponential(60 / 10)  # lambda = 10
srv_distr = sim.Exponential(60 / 12)  # mu = 12

# https://www.salabim.org/manual/Reference.html#environment
app = sim.App(
    trace=False,  # defines whether to trace or not
    random_seed="*",  # if “*”, a purely random value (based on the current time)
    time_unit="minutes",  # defines the time unit used in the simulation
    name="Charging Station",  # name of the simulation
    do_reset=True,  # defines whether to reset the simulation when the run method is called
    yieldless=True,  # defines whether the simulation is yieldless or not
)

# Instantiate and activate the client generator
CarGenerator(name="Electric Cars Generator")

# Instantiate the servers, list comprehension but only 1 server
chargingstations = sim.Resource(
    name="Charging Stations", capacity=N_STATION, monitor=True
)
chargingstations.requesters().length_of_stay.monitor(value=True)

# Execute Simulation
app.run(till=50000)

# Print statistics
# chargingstations.print_statistics()
# chargingstations.requesters().length.print_statistics()
# chargingstations.occupancy.print_statistics()
P0 = 1 - chargingstations.occupancy.mean()
Lq = chargingstations.requesters().length.mean()

print(
    "\n",
    "Probability that the charging station is free: \t",
    P0,
    "\n",
    "Average number of customers in the queue: \t\t",
    Lq,
)

## Example 3

At Bharat charging station, cars arrive according to a Poisson process with an average time of 5 minutes between arrivals. The service time is exponentially distributed with mean time = 2 minutes. On the basis of this information, find out

1. What would be the average queue length?
1. What would be the average number of customers in the queuing system?
1. What is the average time spent by a car in the petrol pump?
1. What is the average waiting time of a car before receiving petrol?

### Solution

Given  

$\lambda = 12/hour$, $\mu = 30/hour$

Average inter arrival time = $\cfrac{1}{\lambda} = 5 minutes = \cfrac{1}{12} = \lambda = 12/hour$
 	 	 	 	 
Average service time =	$\cfrac{1}{\mu} = 2 minutes = \cfrac{1}{30} = \mu = 30/hour$

---
 	 
Average queue length ($L_q$ )  $= \cfrac{12^2}{30(30-12)} = \cfrac{4}{15}$

Average number of customers, ($L_s$)  $ = \cfrac{12}{30-12} = \cfrac{2}{3}$	
 	 
Average time spent at the charging station ($W_s$)  = $\cfrac{1}{\mu-\lambda} = \cfrac{1}{30-12} = $ 3.33 minutes
 	 
Average waiting time of a car before being charged ($W_q$) = $\cfrac{12}{30(30-12)} = $ 1.33 minutes

In [ ]:
# Resource method
# https://www.salabim.org/manual/Modelling.html#the-bank-office-example-with-resources

import salabim as sim


class CarGenerator(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        while True:
            Car()
            self.hold(iat_distr.sample())


class Car(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        self.request(chargingstations)
        self.hold(srv_distr.sample())
        self.release()  # not really required


N_STATION = 1
iat_distr = sim.Exponential(60 / 12)  # lambda = 12/hour
srv_distr = sim.Exponential(60 / 30)  # mu     = 30/hour

# https://www.salabim.org/manual/Reference.html#environment
app = sim.App(
    trace=False,  # defines whether to trace or not
    random_seed="*",  # if “*”, a purely random value (based on the current time)
    time_unit="minutes",  # defines the time unit used in the simulation
    name="Charging Station",  # name of the simulation
    do_reset=True,  # defines whether to reset the simulation when the run method is called
    yieldless=True,  # defines whether the simulation is yieldless or not
)

# Instantiate and activate the client generator
CarGenerator(name="Electric Cars Generator")

# Instantiate the servers, list comprehension but only 1 server
chargingstations = sim.Resource(
    name="Charging Stations", capacity=N_STATION, monitor=True
)
chargingstations.requesters().length_of_stay.monitor(value=True)

# Execute Simulation
app.run(till=50000)

# Print statistics
# chargingstations.print_statistics()

Lq = chargingstations.requesters().length.mean()
Ls = Lq + chargingstations.occupancy.mean()
Wq = chargingstations.requesters().length_of_stay.mean()
Ws = Wq + srv_distr.mean()


print(
    "\n",
    "Average queue length: \t\t\t\t\t",
    Lq,
    "\n",
    "Average number of customers: \t\t\t\t",
    Ls,
    "\n",
    "Average time spent at the charging station: \t\t",
    Ws,
    "\n",
    "Average waiting time of a car before being charged: \t",
    Wq,
)

## Example 4

Enexis is considering to operate a single charging station. Management estimates that customers will arrive at the rate of 15 per hour. The charging station can charge a car at the rate of one every 3 minutes.

Assuming Poisson arrivals and exponential service find

1. Average number in the waiting line.
1. Average number in the system.
1. Average waiting time in line.
1. Average waiting time in the system.

### Solution.
Given  

$\lambda = 15/hour$, $\mu = 20/hour$

Average queue length ($L_q$ )  $= \cfrac{15^2}{20(20-15)} = $ 2.25 cars

Average number of customers, ($L_s$)  $ = \cfrac{15}{20-15} = $ 3 cars	

Average waiting time of a car before being charged ($W_q$) = $\cfrac{15}{20(20-15)} = $ 0.15 hours = 9 minutes

Average time spent at the charging station ($W_s$)  = $\cfrac{1}{\mu-\lambda} = \cfrac{1}{20-15} = $ 12 minutes
 	 

In [ ]:
# Resource method
# https://www.salabim.org/manual/Modelling.html#the-bank-office-example-with-resources

import salabim as sim


class CarGenerator(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        while True:
            Car()
            self.hold(iat_distr.sample())


class Car(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        self.request(chargingstations)
        self.hold(srv_distr.sample())
        self.release()  # not really required


N_STATION = 1
iat_distr = sim.Exponential(60 / 15)  # lambda = 12/hour
srv_distr = sim.Exponential(60 / 20)  # mu     = 30/hour

# https://www.salabim.org/manual/Reference.html#environment
app = sim.App(
    trace=False,  # defines whether to trace or not
    random_seed="*",  # if “*”, a purely random value (based on the current time)
    time_unit="minutes",  # defines the time unit used in the simulation
    name="Charging Station",  # name of the simulation
    do_reset=True,  # defines whether to reset the simulation when the run method is called
    yieldless=True,  # defines whether the simulation is yieldless or not
)

# Instantiate and activate the client generator
CarGenerator(name="Electric Cars Generator")

# Instantiate the servers, list comprehension but only 1 server
chargingstations = sim.Resource(
    name="Charging Stations", capacity=N_STATION, monitor=True
)
chargingstations.requesters().length_of_stay.monitor(value=True)

# Execute Simulation
app.run(till=50000)

# Print statistics
# chargingstations.print_statistics()

Lq = chargingstations.requesters().length.mean()
Ls = Lq + chargingstations.occupancy.mean()
Wq = chargingstations.requesters().length_of_stay.mean()
Ws = Wq + srv_distr.mean()


print(
    "\n",
    "Average queue length: \t\t\t\t\t",
    Lq,
    "\n",
    "Average number of customers: \t\t\t\t",
    Ls,
    "\n",
    "Average waiting time of a car before being charged: \t",
    Wq,
    "\n",
    "Average time spent at the charging station: \t\t",
    Ws,
)

## Example 5

Chhabra Saree Emporium has a single charging station. During the rush hours, cars arrive at the rate of 10 per hour. The average number of customers that can be processed by the charging station is 12 per hour. On the basis of this information, find the following:

1. Probability that the charging station is idle
1. Average number of cars in the queuing system
1. Average time a customer spends in the system
1. Average number of customers in the queue
1. Average time a customer spends in the queue

### Solution.
Given  

$\lambda = 10/hour$, $\mu = 12/hour$

1. Probability of zero unit in the queue ($P_o$) $= 1 - \cfrac{\lambda}{\mu} = 1 - \cfrac{10}{12} = \cfrac{1}{6}$
1. Average number of units in the system ($L_s$) $= \cfrac{\lambda}{\mu-\lambda} = \cfrac{10}{12-10} = $ 5 cars
1. Average waiting time of an arrival in the system ($W_s$) $= \cfrac{1}{\mu-\lambda} = \cfrac{1}{12-10} =$ 30 minutes
1. Average queue length ($L_q$ )  $= \cfrac{\lambda^2}{\mu(\mu-\lambda)} = \cfrac{10^2}{12(12-10)} = \cfrac{25}{6}$ cars
1. Average waiting time of an arrival ($W_q$)  $= \cfrac{\lambda}{\mu(\mu-\lambda)} = \cfrac{10}{12(12-10)} = $ 25 minutes


In [ ]:
# Resource method
# https://www.salabim.org/manual/Modelling.html#the-bank-office-example-with-resources

import salabim as sim


class CarGenerator(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        while True:
            Car()
            self.hold(iat_distr.sample())


class Car(sim.Component):
    def setup(self):
        self.mode.monitor(False)

    def process(self):
        self.request(chargingstations)
        self.hold(srv_distr.sample())
        self.release()  # not really required


N_STATION = 1
iat_distr = sim.Exponential(60 / 10)  # lambda = 10/hour
srv_distr = sim.Exponential(60 / 12)  # mu     = 12/hour

# https://www.salabim.org/manual/Reference.html#environment
app = sim.App(
    trace=False,  # defines whether to trace or not
    random_seed="*",  # if “*”, a purely random value (based on the current time)
    time_unit="minutes",  # defines the time unit used in the simulation
    name="Charging Station",  # name of the simulation
    do_reset=True,  # defines whether to reset the simulation when the run method is called
    yieldless=True,  # defines whether the simulation is yieldless or not
)

# Instantiate and activate the client generator
CarGenerator(name="Electric Cars Generator")

# Instantiate the servers, list comprehension but only 1 server
chargingstations = sim.Resource(
    name="Charging Stations", capacity=N_STATION, monitor=True
)
chargingstations.requesters().length_of_stay.monitor(value=True)

# Execute Simulation
app.run(till=50000)

# Print statistics
# chargingstations.print_statistics()

P0 = 1 - chargingstations.occupancy.mean()
Lq = chargingstations.requesters().length.mean()
Ls = Lq + chargingstations.occupancy.mean()
Wq = chargingstations.requesters().length_of_stay.mean()
Ws = Wq + srv_distr.mean()


print(
    "\n",
    "Probability that the charging station is free: \t",
    P0,
    "\n",
    "Average number of customers: \t\t\t\t",
    Ls,    
    "\n",
    "Average time spent at the charging station: \t\t",
    Ws,    
    "\n",
    "Average queue length: \t\t\t\t\t",
    Lq,
    "\n",
    "Average waiting time of a car before being charged: \t",
    Wq,

)